In [45]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
# pd.set_option('display.max_rows', 100)
import missingno as msno
%matplotlib inline

In [25]:
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

~$DataDictionary.xlsx
DataDictionary.xlsx
game.rar
sample_submission.csv
test.csv
train.csv



In [36]:
# 因为你的输入数据列有混合类型，而PANDAS默认要找到可以使所占用空间最小的类型来储存你的数据。
# low_memory设置为false之后，pandas就不进行寻找，直接采用较大的数据类型来储存。
# train_df = pd.read_csv("../input/train.csv", parse_dates=["transactiondate"])
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")

# train_data=pd.read_csv("../input/train.csv",low_memory=False)

C:\Program Files\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [37]:
print ("Shape Of Train: ",train_df.shape)
print ("Shape Of Test: ",test_df.shape)

Shape Of Train:  (709903, 64)
Shape Of Test:  (177476, 63)


In [28]:
train_df.head(5).transpose()

,0,1,2,3,4
member_id,13599771,1491956,64687987,8446850,58471388
loan_amnt,8875,12500,33600,17000,14000
funded_amnt,8875,12500,33600,17000,14000
funded_amnt_inv,8875,12475,33600,17000,14000
term,36 months,36 months,60 months,60 months,60 months
int_rate,18.25,8.9,18.55,9.71,16.99
installment,321.97,396.92,863.31,358.78,347.87
grade,D,A,E,B,D
sub_grade,D3,A5,E2,B1,D3
emp_title,Inventory/Part time Sup,Bel Brands USA,Assistant Principal,Cook Childrens Medical Center,Cook


In [32]:
print ("Shape Of Positive Train: ",train_df[train_df['acc_now_delinq']==1].shape)
print ("Shape Of Negetive Train: ",train_df[train_df['acc_now_delinq']==0].shape)

Shape Of Positive Train:  (3293, 64)
Shape Of Negetive Train:  (706610, 64)


In [47]:
count_classes = pd.value_counts(train_df['acc_now_delinq'], sort = True).sort_index()
count_classes.plot(kind = 'bar')
plt.title("Fraud class histogram")
plt.xlabel("Class")
plt.ylabel("Frequency")

Text(0,0.5,'Frequency')

In [33]:
missing_df = train_df.isnull().sum(axis=0).reset_index()
missing_df.columns = ['column_name', 'missing_count']
missing_df['missing_ratio'] = missing_df['missing_count'] / train_df.shape[0]
missing_df.loc[missing_df['missing_ratio']>0.999]

,column_name,missing_count,missing_ratio
43,annual_inc_joint,709493,0.999422
44,dti_joint,709495,0.999425
45,verification_status_joint,709493,0.999422


In [42]:
dataTypeDf = pd.DataFrame(train_df.dtypes.value_counts()).reset_index().rename(columns={"index":"variableType",0:"count"})
fig,ax = plt.subplots()
fig.set_size_inches(20,5)
sn.barplot(data=dataTypeDf,x="variableType",y="count",ax=ax,color="#34495e")
ax.set(xlabel='Variable Type', ylabel='Count',title="Variables Count Across Datatype")

[Text(0,0.5,'Count'),
 Text(0.5,0,'Variable Type'),
 Text(0.5,1,'Variables Count Across Datatype')]

In [48]:
missingValueColumns = train_df.columns[train_df.isnull().any()].tolist()
msno.bar(train_df[missingValueColumns],\
            figsize=(20,8),color="#34495e",fontsize=12,labels=True,)